# Semelhança de documentos

A função desse notebook é criar representações para o texto dos documentos, exibir textos semelhantes e com isso avaliar as representações.
Os textos semelhantes são um teste para a representação do texto: se os textos mostrados não são semelhantes, então a representação não é boa; se são semelhantes, então a representação um pouco melhor.

In [23]:
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
# Importa arquivos
# Descomente pra usar no Google Colab
# from google.colab import files
# import os.path

laws_file = 'leis.json'

# if (not os.path.isfile(arq_leis)):
#     uploaded = files.upload()

In [25]:
laws = pd.read_json(laws_file)
laws.drop(['documento'], inplace=True, axis=1)
print(laws.info())
print(laws.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titulo     6033 non-null   object
 1   categoria  6033 non-null   object
 2   resumo     6033 non-null   object
 3   texto      6033 non-null   object
dtypes: object(4)
memory usage: 188.7+ KB
None
titulo       6033
categoria       8
resumo       4961
texto        6029
dtype: int64


In [26]:
laws

,titulo,categoria,resumo,texto
0,"DECRETO Nº 8854, de 28 de fevereiro de 2013.",Decretos,DELEGA COMPETÊNCIA À SECRETARIA MUNICIPAL DE P...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
1,"DECRETO Nº 8849, de 25 de fevereiro de 2013.",Decretos,ABRE CRÉDITO SUPLEMENTAR AO ORÇAMENTO DO MUNIC...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
2,"DECRETO Nº 8853, de 27 de fevereiro de 2013.",Decretos,NOMEIA MEMBROS DO CONSELHO MUNICIPAL DE DESENV...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
3,"DECRETO Nº 8967, de 17 de julho de 2013",Decretos,ALTERA O QUADRO DE DETALHAMENTO DE DESPESA DO ...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
4,"DECRETO Nº 8982, de 30 de julho de 2013",Decretos,AUTORIZA O FUNCIONAMENTO DE ESTABELECIMENTOS C...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
...,...,...,...,...
6028,RESOLUÇÃO Nº 125/1980,Resoluções,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Faço saber que a Câmara Municipal aprovou e eu...
6029,RESOLUÇÃO Nº 403/2003,Resoluções,AUTORIZA A MESA DIRETIVA DO PODER LEGISLATIVO ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6030,RESOLUÇÃO Nº 492/2014,Resoluções,INSTITUI A SEGUNDA SEMANA DO MÊS DE AGOSTO EM ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6031,RESOLUÇÃO Nº 382/2001,Resoluções,CRIA A MEDALHA VEREADOR DIVAL FIGUEIREDO MACHA...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."


In [27]:
# Exemplo de texto de lei
laws.loc[len(laws)-1, 'texto']

'A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:\n\nArt. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:\n\n"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.\n\n§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."\n\n"Art. 33 Compete, privativamente, ao Vice-Presidente:"\n\n"Art. 36 ...\n\nI - ...\n\ne) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presiden

## Comparando documentos: representação e calculo de similaridade

Para comparar quão parecido são dois documentos, primeiro temos que transformar estes documentos para uma representação que o computador consiga calcular alguma coisa a respeito. Existem alguns métodos para isto. Neste notebook temos 3: TF, TF-IDF e vetores de palavras. Para calcular a similaridade, também existem alguns métodos diferentes. Utilizamos similaridade do cosseno.

### Term Frequency (TF)

A primeira representação construída é bastante ingênua:
apenas conta a quantidade de vezes que cada palavra apareceu em cada texto e atribuir um vetor pra esse texto.
Cada posição do vetor é uma palavra e cada valor representa quantas vezes essa palavra apareceu no dado texto.
Todos os textos, portanto, são representados por uma matriz de dimensões _m_ x _n_,
 onde _m_ é o número de textos e _n_ é o número de palavras únicas (tamanho do vocabulário).

In [28]:
from scripts.parsers import clean_text
laws['texto_limpo'] = laws['texto'].apply(clean_text)

In [ ]:
# Generates document matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
tf_representation = vectorizer.fit_transform(laws['texto_limpo'])
tf_representation

Com a matriz de documentos ~literalmente~ em mãos, vamos calcular a similaridade entre dois textos.
A similaridade é calculada pela similaridade do cosseno (ver algebra linear).

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

cos_sim_matrix = cosine_similarity(tf_representation, dense_output=True)
cos_sim_argsort = np.argsort(cos_sim) # sorts ascending, per row, the indexes of the documents according to their cossine similarity

In [31]:
most_similar_indexes_tf = cos_sim_argsort[:,-2] # -1 is the same text
tf_similarities = [cos_sim_matrix[i, ind] for i, ind in enumerate(most_similar_indexes_tf)] 

In [36]:
def print_laws(original_law_index, compared_law_index: int):
    print(f'- - - LEI: {original_law_index}: - - -\n\n')
    print(laws.loc[original_law_index, 'texto'])
    print(f'\n\n- - - LEI COMPARADA: {compared_law_index} - - -\n\n')
    print(laws.loc[compared_law_index, 'texto'])

In [18]:
max_sim_overall = np.max(tf_similarities)
print(f'Maximum similarity between two laws: {max_sim_overall}')

original_law_index = np.argmax(tf_similarities)
most_similar_law_index = most_similar_index[original_law_index]

print_laws(original_law_index, most_similar_law_index)

Maximum similarity between two laws: 1.0000000000000013
- - - LAW: 5949: - - -


A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do Art. 70, inciso VII, da Lei Municipal nº37de 05 de abril de 1990, e Artigos 72, inciso III e 138 § 2º, do Regimento Interno, Promulga a seguinte Resolução:

Art. 1ºFicam reajustados em 12% (doze por cento), os valores dos vencimentos dos servidores da Câmara Municipal de Feira de Santana, referentes ao mês de maio do corrente ano.

Art. 2ºAs despesas decorrentes desta Resolução correrão por conta de verba orçamentária desta Câmara.

Art. 3ºEsta Resolução entrará em vigor na data da sua publicação e seus efeitos a partir de 1º de maio de 1996, revogadas as disposições em contrário.

MESA DIRETIVA DA CÂMARA MUNICIPAL DE FEIRA DE SANTANA, EM 18 DE JUNHO DE 1996.

JOSÉ FLANTILDES RIBEIRO DE OLIVEIRA
Presidente

JOSÉ MARCONE PAULO DE SOUSA
1º Vice-Presidente

SEVERINO SOARES
1º Secretário

NANTES BELLAS VIEIRA
2º Secretário


- - - COMP

As leis 5949 e 6026 são idênticas.
Opa! Lei 13 e Lei 118 são a mesma lei, com 3 dias de diferença. Por que existe isso?

### TF-IDF
Outra representação possível para os textos é TF-IDF. 
Ainda contamos a frequência de cada termo (TF), porém ponderamos esta frequência pela raridade da palavra, medida pela frequência inversa que ela aparece nos documentos (Inverse Document Frequency). 
Ou seja, quanto mais rara é a palavra no corpus, mais ela caracteriza o texto em que ela aparece, maior será o peso dela.

In [20]:
transformer = TfidfTransformer()
tfidf_representation = transformer.fit_transform(tf_representation)

cos_sim_tfidf = cosine_similarity(tfidf_representation, dense_output=True)
cos_sim_tfidf_sorted_idxs = np.argsort(cos_sim_tfidf)

tfidf_representation

<6033x30209 sparse matrix of type '<class 'numpy.float64'>'
	with 625873 stored elements in Compressed Sparse Row format>

In [21]:
most_similar_law_idx = cos_sim_tfidf_sorted_idxs[original_law_index, -2]
tfidf_similarity = cos_sim_tfidf[original_law_index, most_similar_law_idx]

print(f'Given same law, the highest similarity using TF-IDF is: {tfidf_similarity}')
print_laws(original_law_index, most_similar_law_idx)

Given same law, the highest similarity using TF-IDF is: 1.0000000000000002
- - - LAW: 5949: - - -


A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do Art. 70, inciso VII, da Lei Municipal nº37de 05 de abril de 1990, e Artigos 72, inciso III e 138 § 2º, do Regimento Interno, Promulga a seguinte Resolução:

Art. 1ºFicam reajustados em 12% (doze por cento), os valores dos vencimentos dos servidores da Câmara Municipal de Feira de Santana, referentes ao mês de maio do corrente ano.

Art. 2ºAs despesas decorrentes desta Resolução correrão por conta de verba orçamentária desta Câmara.

Art. 3ºEsta Resolução entrará em vigor na data da sua publicação e seus efeitos a partir de 1º de maio de 1996, revogadas as disposições em contrário.

MESA DIRETIVA DA CÂMARA MUNICIPAL DE FEIRA DE SANTANA, EM 18 DE JUNHO DE 1996.

JOSÉ FLANTILDES RIBEIRO DE OLIVEIRA
Presidente

JOSÉ MARCONE PAULO DE SOUSA
1º Vice-Presidente

SEVERINO SOARES
1º Secretário

NANTES BELLAS VIEIRA
2º Secr

Mostram a mesma lei. O que faz sentido, já que as leis são idênticas.

In [34]:
most_similar_indexes_tfidf = cos_sim_tfidf_sorted_idxs[:,-2]

same_result = (most_similar_indexes_tf == most_similar_indexes_tfidf)
print(same_result)
print(f"Agrees on {sum(same_result) / len(same_result) * 100}% of the results")

[False  True  True ...  True  True False]
Agrees on 40.44422343775899% of the results


A lei mais semelhante de acordo com TF e TF-IDF é a mesma 40% das vezes.

Vamos dar uma olhada em algumas leis onde os resultados diferem, para ter uma intuição sobre qual representação é melhor para as leis.

Vamo sortear indices aleatorios desse vetor e ler as leis que eles representam e as similaridades

In [38]:
different_result_indexes = [i for i, _ in enumerate(same_result) if not same_result[i]]
comparisons_count = 10
drafted_indexes = np.random.randint(0, high=len(different_result_indexes)-1, size=comparisons_count)
drafted_laws = [different_result_indexes[i] for i in drafted_indexes]
print(drafted_laws)

[3958, 1845, 2942, 1569, 2097, 4906, 2067, 2713, 1197, 61]


In [40]:
for i in drafted_laws:
    print(f'\n\nCOMPARACAO UTILIZANDO TF:\n\n')
    print_laws(i, most_similar_indexes_tf[i])
    print('\n\nCOMPARACAO UTILIZANDO TF-IDF:\n\n')
    print_laws(i, most_similar_indexes_tfidf[i])

 na Rua Cônego Olímpio, nº 83, Distrito de Humildes, neste Município.

Art. 2ºEsta Lei entrará em vigor na data sua publicação, revogadas as disposições em contrário.

Gabinete do Prefeito, 16 de maio de 2017.

JOSÉ RONALDO DE CARVALHO
PREFEITO MUNICIPAL

MARIO COSTA BORGES
CHEFE DE GABINETE DO PREFEITO

CLEUDSON SANTOS ALMEIDA
PROCURADOR GERAL DO MUNICÍPIO

PUBLICADO NO DIÁRIO OFICIAL ELETRÔNICO DIA 18 DE MAIO DE 2017.


COMPARACAO UTILIZANDO TF-IDF:


- - - LEI: 2097: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, Faço Saber que a Câmara Municipal, através do Projeto de Lei nº 44/2017, de autoria do Edil José Carneiro Rocha, decretou e eu sanciono a seguinte Lei:

Art. 1ºFica considerado de utilidade pública o INSTITUTO TENDA DA PAZ, com sede na Rua Picos, nº 33, Bairro da Queimadinha, neste Município de Feira de Santana.

Art. 2ºEsta Lei entrará em vigor na data de sua publicação, revogadas as disposições em contrário.

Gabinete do Prefeito, 02 de maio de 2017.



### Lei 1018
Lei 1018 é sobre proibição de homenagens a condenados por corrupção. Count trouxe uma lei sobre tornar uma associação pública. TF-IDF trouxe uma lei sobre evento de comemoração de adoção animal. Todas duas erraram. TF-IDF chegou mais perto? Difícil dizer

### Lei 5776
Lei 5776 sobre pagamento servidor público. Count trouxe: leitura da bíblia na abertura da câmara. TF-IDF: aposentadoria diretor valor vencimento etc. Ambas as leis parecem ter sido trazidas como semelhantes pq dos nomes próprios contidos nas leis.

### Lei 2789
Lei 2789 (mil anos da revolução francesa) sobre obrigatoriedade de um servidor formado em primeiros socorros em escolas. Count: faço saber inkaba instituto de karate. TF-IDF: faço saber associação estrela jaco. Novamente as semelhanças são os nomes próprios nas leis.

### Lei 1772
Lei 1772: faço saber sindicato trabalhadores rurais. Count: faço saber associação profissionais sexo. TF-IDF: faço saber associação pequenos agricultures apaeb. A rua da sede é a mesma da lei comparada. 

TF-IDF se saiu melhor nessa. Os nomes das pessoas em Count eram os mesmos da Lei, mas em TF-IDF não. O fator decisivo aqui foi o nome da rua, que era o mesmo. Ponto pra TF-IDF.

### Lei 503
Lei 530: faço saber igreja ministerio pentecostal fogo gloria. rua volta redonda bairro campo limpo.
Count: faço saber instituto nobre sede rodovia br km cis. nomes das pessoas iguais.
TF-IDF: faço saber igreja evangelica pentecostal monte carmelo rua espassonavel bairro george americo. prefeitos diferentes.

### Lei 4810
Lei: comenda. nomes: godofredo rebell figueiredo filho, raymundo luiz oliveira lopes. 
Count: comenda. nomes: godofredo rebello figueiredo filho, nilton bellas vieira.
TF-IDF: comenda. nomes: godofredo rebello figueiredo filho, raimundo antonio carneiro pinto.

As duas acertaram. TF-iDF nome mais parecido? Não deve influenciar...

### Lei 5238 
Lei 5238: promulgação de novas vias públicas. A via por TF-IDF passa por mais ruas semelhantes.

### Lei 5383
Lei promulga academia de ginástica. Count: promulga empresas serviço funerario. TF-IDF: promulga novos aparelhos de ginástica.

### Outras
As outras leis eram: _visualizar legislativo ba_. Ambas trouxeram textos idênticos.

Ok! Massa! Funciona!

Pelos resultados acima, TF-IDF se saiu melhor. 
Inclusive pra retornar semelhança por nomes de ruas e de bairros, que é o que a gente quer pras buscas.

Mesmo com 28k features, o resultado foi bastante rápido. 
Caso tivessemos um corpus maior, poderíamos ainda usar PCA pra reduzir as dimensões e ainda assim calcular a similaridade mantendo as relações entre os documentos.

## Outras opções
### Indexar
Há outras formas de indexar os documentos e de recuperar, também simples. Uma outra forma de indexar, por exemplo, é fazer um vetor pra cada palavra contando as palavras vizinhas. E depois, o vetor do documento seria a soma dos vetores das palavras. É uma forma interessante porque pode gerar visualizações interessantes entre a similaridade das palavras. Por exemplo, no corpus das Leis Municipais, a quais palavras EDUCAÇÃO mais se assemelha? Ou SAÚDE? Etc.

Outra forma é contar n-gramas - por exemplo, bi-gramas: duas palavras juntas formando um token. Dessa forma, você possui uma matriz maior e de certa forma uma relação entre a sequencialidade das palavras, que pode ser útil pra nomes de pessoas e bairros, como citado acima.

### Recuperar
Outra forma de recuperar é por local sensitive hashing. Divide em vários planos múltiplas vezes e retorna os resultados que estão na mesma região da query. No entanto, o corpus não é grande o suficiente pra precisar essa estratégia, que é mais pra grandes corpora. O método acima (calcular a simlaridade cosseno e retornar os maiores valores) é rápido o suficiente pra parecer instantâneo. Talvez com uma demanda mais alta pelo servidor venha a necessidade de aumentar a velocidade da busca, porém por enquanto não é o caso. Mais sobre recuperação: Google lançou novo método e uma lib pra isso agora, dia 28 de Julho: https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html

### Avaliação
Com múltiplas formas de indexar e recuperar vem o dilema: como avaliar se uma é melhor que a outra? Repetir o processo acima pra todas as opções? Isto é, mostrar N melhores resultados e comparar manualmente? Ou colocar labels em algumas leis? Ex: essa lei trata disso, com tais entidades. Checar formas de avaliação. Se tivesse em produção, podia avaliar por CTR por ex, mas não é o caso

In [27]:
# Descrevendo a palavra pelos seus vizinhos
texto_limpo = ' '.join(leis['texto_limpo'].tolist())
texto_limpo = texto_limpo.split()
palavras_unicas = set(texto_limpo)
print(len(texto_limpo))
len(palavras_unicas)

972616


28875

In [28]:
# Transformar nosso set pra um dicionário de indices
idx_palavras = {}
for i, palavra in enumerate(palavras_unicas):
    idx_palavras[palavra] = i

In [29]:
# Criando matriz do embedding
vetores = np.zeros((len(palavras_unicas), len(palavras_unicas)), dtype=np.int16)
vetores

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [30]:
# Definir ate qual distancia utilizar
vizinhanca = 2
for idx, palavra in enumerate(texto_limpo):
    for i in range(1, vizinhanca):
        pal_vizinha = texto_limpo[idx+i]

        idx_pal = idx_palavras[palavra]
        idx_vizinha = idx_palavras[pal_vizinha]

        vetores[idx_pal, idx_vizinha] += 1
        vetores[idx_vizinha, idx_pal] += 1
    if (idx == len(texto_limpo) - vizinhanca):
        break
vetores

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [31]:
# Matriz tem que ser espassa, se não na hora de calcular
# similaridade gasta muita memoria

from scipy.sparse import csr_matrix
vetores = csr_matrix(vetores)
vetores

<28875x28875 sparse matrix of type '<class 'numpy.int16'>'
	with 658329 stored elements in Compressed Sparse Row format>

In [32]:
# Ok, temos nossas representacoes pras palavras
# Como um teste de sanidade, vamos ver 
# as palavras mais proximas de 10 palavras aleatorias
cos_sim_palavras = cosine_similarity(vetores, dense_output=False)
cos_sim_palavras


<28875x28875 sparse matrix of type '<class 'numpy.float64'>'
	with 76446099 stored elements in Compressed Sparse Row format>

In [33]:
# palavras_semelhantes = np.asanyarray(cos_sim_palavras)
# palavras_semelhantes = np.argsort(palavras_semelhantes, axis=1)
# palavras_semelhantes = np.sort(cos_sim_palavras)
palavras_semelhantes = np.argsort(cos_sim_palavras[0].toarray())
palavras_semelhantes

array([[14437, 18921, 18920, ..., 14528, 14974,     0]])

In [34]:
idx_palavras_rand = np.random.randint(len(palavras_unicas), size=10)

def mostra_palavras_semelhantes(idx: int, semelhantes: list, lista_palavras: list):
    pal = lista_palavras[idx]
    print(f'Lista de palavras semelhantes a {pal} - {idx}:')
    for i in semelhantes:
        pal = lista_palavras[i]
        print(f'{pal} - {i}')
    print('\n- - - - - \n\n')

lista_palavras = list(idx_palavras.keys())
for idx in idx_palavras_rand:
    palavras_semelhantes = np.argsort(cos_sim_palavras[idx].toarray())
    semelhantes = palavras_semelhantes[0][-10:-1]
    mostra_palavras_semelhantes(idx, semelhantes, lista_palavras)

Lista de palavras semelhantes a funerais - 3755:
rela - 25787
oe - 18886
volumosa - 28313
diligenciando - 3698
equipando - 22118
aerofotogrametria - 22416
sondagem - 13553
portico - 2178
adjudicacoes - 13209

- - - - - 


Lista de palavras semelhantes a eeg - 4993:
concorrentemente - 744
discordancia - 27422
consorciar - 9462
estimulaa - 10918
instrumen - 21973
usada - 13383
paramentos - 12168
estimulacao - 1754
televisores - 7262

- - - - - 


Lista de palavras semelhantes a estomologia - 7844:
laboratorial - 22997
terapeutico - 3229
turma - 12594
compondo - 3565
imagem - 11763
radioisotopos - 1179
neonatal - 26241
socioterritorial - 27798
estomologia - 7844

- - - - - 


Lista de palavras semelhantes a digitalizada - 16781:
base - 16268
regras - 4471
implantar - 4372
restituicao - 4335
aeronaves - 4581
visual - 5228
comprometer - 20083
aero - 12379
ifr - 4252

- - - - - 


Lista de palavras semelhantes a providencias - 8126:
estatuindo - 7324
repassa - 11663
procuracoes - 1967
recupe

Neste corpus, pra algumas palavras, a hipótese distribucional parece funcionar bem, pra outras nem tanto, pra outras não funciona.

Semelhantes a "outorgar" temos: "permutar", "editar", "contratar", "doar", "conceder", "dispensar", "celebrar", "subscrever", "proibir". Embora a semântica (significado) não seja necessariamente próxima, todas as palavras são verbos, então a sintaxe é próxima. Semelhantes a "ibitita": "axixa", "ibirarema", "peritoro", "piracaia", "igarata", "erechim", "itaperuna", "piata", "vandinha". Todos parecem nomes de locais.

Existem casos horríveis. Semelhantes à "coesao" temos: "sedeso", "his", "ctps", "zeis", "pnas", "cgfmhis", "snhis", "acemas". O que significam essas palavras? Talvez seja útil melhorar a qualidade do pré-processamento pra melhorar na indexação. Semelhantes à "separando" temos: "agrossilvopastoris", "cemiteriais", "solidos", "molhados", "domiciliares", "volumosos", "baldios", "antecedencia", "dimensao".

Há casos mistos. Semelhantes `a "trasporte" (note o erro) temos: "meia" (talvez meia passagem?), "transporte" (a palavra correta aparece em segundo), "roletas", "vala" (?), "trafegos", "convencional" (?), "edificar" (?), "passageiros", "fretado".

Talvez o corpus seja pequeno demais pra encontrar as relações entre as palavras só contando? Há de se testar se não é melhor então trabalhar com vetores de palavras, mesmo aprendidos em um corpus pequeno. Segundo o paper "Don't count, predict! a systematic comparison of context-counting vs. context-predicting semantic vectors (2014) - Baroni, Dinu, Kruszeweski", predizer é melhor que contar. Há de se testar se neste nosso contexto isso também se verifica.

Por hora, vamos testar se a busca melhora ou não utilizando as palavras. Então vamos construir a representação das leis.

# Construindo representação das leis com base na hipotese distribucional

In [35]:
# Cada lei vai ser a soma dos vetores de suas palavras
# Usando np.zeros gasta muita memoria, mas csr_matrix eh muito lento
leis_vetores_palavras = np.zeros((len(leis['texto_limpo']), vetores.shape[1]))
for idx, lei in enumerate(leis['texto_limpo']):
    for palavra in lei.split():
        idx_palavra = idx_palavras[palavra]
        leis_vetores_palavras[idx] += vetores[idx_palavra]
leis_vetores_palavras

array([[ 1.,  0.,  4., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 2., 12.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  6.,  0., ...,  0.,  0.,  0.],
       [ 0., 21.,  0., ...,  0.,  0.,  0.],
       [ 0., 33., 17., ...,  0.,  0.,  0.]])

In [36]:
vetores_similaridade = cosine_similarity(leis_vetores_palavras)

In [37]:
mais_semelhantes_vetores = [idx for idx in np.argsort(vetores_similaridade)[:,-2]]

In [42]:
# Diferenca de vetores pra TFIDF eh bem maior.
# Vamos samplear algumas dessas diferencas e 
# mostrar uma lei q tanto TFIDF como Count erraram
diferencas_TFIDF = (mais_semelhantes_vetores != max_sim_idx_tfidf)
sum(diferencas_TFIDF)

5163

In [43]:
sorteadas = np.random.randint(len(diferencas_TFIDF), size=10)
for i in sorteadas:
    if diferencas_TFIDF[i]:
        print(f'\n\nLei numero {i}\nPAR UTILIZANDO VETOR:\n\n')
        print_lei_mais_sem(i, mais_semelhantes_vetores[i])
        print('\n\nPAR UTILIZANDO TF-IDF:\n\n')
        print_lei_mais_sem(i, max_sim_idx_tfidf[i])



Lei numero 3366
PAR UTILIZANDO VETOR:


- - - LEI COMPARADA 3366: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, FAÇO saber que a Câmara Municipal, através do Projeto de Lei nº 5/2014, de autoria da Edil Gerusa Maria Bastos Sampaio, decretou e eu sanciono a seguinte Lei:

Art. 1ºFica considerado de utilidade pública o PROJETO IRMÃO SOLIDÁRIO - PROISO, com sede localizada à Rua Estrela do Sul, nº 200, Conjunto Stella Maria, no Bairro Muchila I, neste Município de Feira de Santana.

Art. 2ºEsta Lei entrará em vigor na data de sua publicação, revogadas as disposições em contrário.

Gabinete do Prefeito, 19 de Março de 2014.

JOSÉ RONALDO DE CARVALHO
Prefeito

MÁRIO COSTA BORGES
Chefe de Gabinete do Prefeito

Autor: Gerusa Maria Bastos Sampaio


- - - LEI MAIS SEMELHANTE 2145 - - -


O PREFEITO MUNCIPAL DE FEIRA DE SANTANA, Estado da Bahia, FAÇO saber que a Câmara Municipal, através do Projeto de Lei nº 135/2013, de autoria da Edil Gerusa Maria Bastos Sampaio, decreto

Parece que TF-IDF é um pouco melhor na comparação de leis, pq traz resultados mais relevantes quando comparados nome de bairros e ruas. No entanto, a forma vetorizada parece ser boa pra reconhecer formatos da Lei em geral, uma especie de POS, reconhecendo que existe uma entidade alí ou verbo etc. Ao menos foi minha impressão. Cabe mais investigação a respeito.

Vale salientar que a qualidade dos vetores parece não estar tão boa. Vide a semelhança de palavras. Como fazer pra consertar isso? Seria muito interessante corrigir isso pra ver as palavras mais semelhantes à educação, saúde etc e também pra visualizar com tsne os clusters gerados a partir daí.